In [44]:
import pandas as pd
import logging 
import numpy as np
import re 
from IPython.display import display
from sqlalchemy import MetaData, Table
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from unidecode import unidecode
from dateutil import parser

In [ ]:
# from 31 july 
# try to get to the bottom of the 'lat' thing, I have a list of 10 but am confused about how to take them out 

In [28]:
# LOGGING 
print('Started cleaning_store_details')

#creating instance of dataextractor 
instance = DataExtractor()

#retrieving the data from the stores API - this takes around 90 seconds 
df = instance.retrieve_stores_data()  

Started cleaning_store_details
retrieve_stores_data is working
read_api_key is working
read_api_key is done
Skipped Stores DataFrame:
Empty DataFrame
Columns: []
Index: []


In [29]:
#making a back up of the dataframe 
df_backup = df.copy()

In [30]:
# run this to reset the df 
df = df_backup.copy()

In [46]:
df = df_backup.copy()



num_rows = df.shape[0]
print(f"Number of rows store data before cleaning: {num_rows}")

# STEP 1 make the webstore have prooper entries 

# logging 
# before = df.iloc[0]
# display(before)

# Define the index of the record to update
index_to_update = 0

# Define the values to replace 'N/A' with for the specific record
df.loc[index_to_update, 'address'] = 'online'
df.loc[index_to_update, 'longitude'] = 1
df.loc[index_to_update, 'lat'] = 1
df.loc[index_to_update, 'locality'] = 'online'
df.loc[index_to_update, 'latitude'] = 1

# logging
# after = df.iloc[0]
# display(after)


# STEP 2 remove garbage records and NULL from lat 

# Define the regex pattern to match invalid lat values
pattern = r'^[A-Za-z0-9]+$'

# Create a boolean mask for rows to keep: rows that do not match the pattern and are not 'NULL'
mask = ~df['lat'].str.contains(pattern, na=False) & ~df['lat'].isin(['NULL'])

display(df[~mask])

# Filter the DataFrame using the mask
df = df[mask]

num_rows = df.shape[0]
print(f"Number of rows store data after dropping lat column: {num_rows}")

#STEP 3 cleaning staff numbers

# Define the regex pattern to match digits only
pattern = r'^\d+$'

# Create a boolean mask for rows that are not digits
mask = ~df['staff_numbers'].str.match(pattern, na=False)

# Display the rows that will be converted to 0
print("Rows that will be converted to 0:")
display(df[mask])

# Convert non-digit values to 0
df.loc[mask, 'staff_numbers'] = 0

# Convert the staff_numbers column to integer
df['staff_numbers'] = df['staff_numbers'].astype(int)

# STEP 4, cleaning continents 

# replacing incorrect spellings of continents 
continent_replacements = {
    'eeEurope': 'Europe',
    'eeAmerica': 'America'
}

df['continent'] = df['continent'].replace(continent_replacements)

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning continent: {num_rows}")

# STEP 5, cleaning locality 

# filtering out items in locality that aren't real place names or NULL 
pattern = r'^[a-zA-Z\s-]+$'
locality_mask = df['locality'].str.match(pattern, na=False) | (df.index == 0)

# Capture the rows that will be dropped
rows_dropped_by_locality = df[~locality_mask | df['locality'].replace('NULL', np.nan).isna()]
print("Rows that will be dropped during locality cleaning:")
print(rows_dropped_by_locality)

# Apply the locality mask
df = df[locality_mask]

# Replace 'NULL' with np.nan and drop rows where 'locality' is NaN
df['locality'] = df['locality'].replace('NULL', np.nan)
df = df.dropna(subset=['locality'])

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning locality: {num_rows}")

# STEP 5, cleaning opening_date 

num_rows = df.shape[0]
print(f"Number of rows before opening_date cleaning: {num_rows}")

# Initialize a list to store invalid dates
invalid_dates_list = []

# Function to parse dates and standardize format
def parse_date(date_str):
    try:
        # Attempt to parse the date string to a datetime object
        dt = parser.parse(date_str)
        # Convert to the desired format (YYYY-MM-DD)
        return dt.strftime('%Y-%m-%d')
    except (parser.ParserError, ValueError):
        # Append invalid date to the list
        invalid_dates_list.append(date_str)
        return np.nan  # Return NaN for invalid dates

# Apply the function to the 'date_of_birth' column
df['opening_date'] = df['opening_date'].apply(parse_date)

# Identify rows that would be null after conversion
invalid_rows = df[df['opening_date'].isna()]

display("Rows that would be converted to NULL:")
display(invalid_rows)

# Drop rows with NaN (invalid dates)
df_cleaned = df.dropna(subset=['opening_date'])

display("\nList of invalid dates:")
display(invalid_dates_list)

num_rows = df.shape[0]
print(f"Number of rows after opening_date cleaning: {num_rows}")

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning: {num_rows}")

#returning the dataframe 


Number of rows store data before cleaning: 451


,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
63,63,6FWDZHD7PW,1ZVU03X2P6,13KJZ890JH,9IBH8Y4Z0S,NRQKZWJ9OZ,BIP8K8JJW2,ZCXWWKF45G,QP74AHEQT0,1CJ5OAU4BR,YELVM536YT,QMAVR5H3LD
172,172,UBCIFQLSNY,Q1TJY8H1ZH,2XE1OWOC23,1T6B406CI8,QIUU9SVP51,SKBXAXF5G5,7AHXLXIUEF,O0QJIRC943,3ZR3F89D97,FP8DLXQVGH,LU3E036ZD9
217,217,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
231,231,RC99UKMZB2,2YBZ1440V6,OXVE5QR07O,6LVWPU1G64,Y8J0Z2W8O9,2429OB3LMM,0OLAK2I6NS,50IB01SFAZ,L13EQEQODP,HMHIFNLOBN,5586JCLARW
333,333,X349GIDWKU,O7NF1FZ74Y,VKA5I8H32X,RX9TCP2RGB,ISEE8A57FE,74BY7HSB6P,A3PMVM800J,0RSNUU3DF5,J3BPB68Z1J,F3AO8V2LHU,GFJQ2AAEQ8
381,381,ZBGB54ID4H,SKO4NMRNNF,LACCWDI0SB,CQMHKI78BX,T0R2CQBDUS,GT1FO6YGD4,GMMB02LA9V,B4KVQB3P5Y,AJHOMDOHZ4,OH20I92LX3,SLQBD982C0
405,405,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
414,414,XTUAV57DP4,ID819KG3X5,A3O5CBWAMD,RY6K0AUE7F,TUOKF5HAAQ,FRTGHAA34B,13PIY8GD1H,X0FE7E2EOG,AE7EEW4HSS,OYVW925ZL8,XQ953VS0FG
437,437,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
447,447,K0ODETRLS3,K8CXLZDP07,UXMWDMX1LC,3VHFDNP8ET,9D4LK7X4LZ,D23PCWSM6S,36IIMAQD58,NN04B3F6UQ,JZP8MIJTPZ,B3EH2ZGQAV,1WZB1TE1HL


Number of rows store data after dropping lat column: 441
Rows that will be converted to 0:


,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
31,31,"Flat 69\nSuzanne walk\nEast Michelle\nE80 8HS,...",52.68333,None,East Dereham,EA-24B31935,J78,2012-11-09,Outlet,0.93333,GB,Europe
179,179,"Girschnerweg 163\n93597 Angermünde, Dahlem",52.4581,None,Dahlem,DA-ACC520AE,30e,1994-03-07,Local,13.28702,DE,Europe
248,248,"7430 Howe Extensions Suite 299\nKellyside, WA ...",39.71734,None,Sicklerville,SI-C489938D,80R,1994-02-28,Outlet,-74.96933,US,America
341,341,Studio 8\nLydia groves\nNorth Hilarymouth\nIV4...,50.79205,None,Southsea,SO-B5B9CB3B,A97,2018-05-08,Super Store,-1.08593,GB,Europe
375,375,"Salzstraße 1/9\n74209 Bad Freienwalde, Charlot...",52.53048,None,Charlottenburg-Nord,CH-99475026,3n9,1995-03-05,Local,13.29371,DE,Europe


Number of rows store data after cleaning continent: 441
Rows that will be dropped during locality cleaning:
Empty DataFrame
Columns: [index, address, longitude, lat, locality, store_code, staff_numbers, opening_date, store_type, latitude, country_code, continent]
Index: []
Number of rows store data after cleaning locality: 441
Number of rows store data after cleaning: 441
Number of rows before opening_date cleaning: 441


'Rows that would be converted to NULL:'

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent


'\nList of invalid dates:'

[]

Number of rows after opening_date cleaning: 441


In [45]:
# STEP 6 clean date of birth 



Number of rows before opening_date cleaning: 441


'Rows that would be converted to NULL:'

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent


'\nList of invalid dates:'

[]

Number of rows after opening_date cleaning: 441


In [ ]:
# Create function to text data 
def clean_text_data(connection, table_name, column_name):
    clean_text_sql = f"""
    UPDATE "{table_name}"
    SET "{column_name}" = NULL
    WHERE CAST("{column_name}" AS TEXT) !~ '^[A-Za-z_]+$';
    """
    connection.execute(text(clean_text_sql))

WORKING OUTS BELOW 

In [21]:
df['locality'].unique()

array(['N/A', 'High Wycombe', 'Landshut', 'Westbury', 'Belper',
       'Gainsborough', 'Rutherglen', 'Stuttgart', 'Kaukauna', 'Hartley',
       'Siegburg', 'Devizes', 'Crystal Lake', 'Halstenbek', 'Lancing',
       'Newbury', 'Lymington', 'Chapletown', 'Barnet', 'East Dereham',
       'Poulton-le-Fylde', 'Bushey', 'Ehingen', 'Aberdeen', 'Surbiton',
       'Inverness', 'Weimar', 'Penzance', 'Albstadt', 'Hechingen',
       'Mexborough', 'Arbroath', 'Cowes', 'Clacton-on-Sea', 'Boston',
       'Charlottenburg-Nord', 'Bretten', 'Sun City Center', 'Karben',
       'Martinsburg', 'Oschatz', 'Kingston upon Hull', 'Leyland',
       'Venice', 'Schleswig', 'Eppingen', '9IBH8Y4Z0S', 'Losheim',
       'Burscheid', 'Southsea', 'Porterville', 'Walsrode', 'Exeter',
       'Thetford', 'Morningside Heights', 'Searcy', 'Radevormwald',
       'Stade', 'Mira Mesa', 'Strood', 'Verl', 'Wesseling', 'Pfullingen',
       'Bensheim', 'Walton-on-the-Naze', 'Friedberg', 'Winsford',
       'Newburgh', 'Selby', 'Obe

In [14]:
# Locate the row(s) where 'lat' is 'N/A'
na_rows = df.loc[df['lat'] == 'N/A']

null_rows = df.loc[df['lat'] == 'NULL']


# Print the 'N/A' row(s) for visual inspection
print("Rows where 'lat' is 'N/A':")
display(na_rows)
display(null_rows)

Rows where 'lat' is 'N/A':


,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe


,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent


In [13]:
# Define the regex pattern to match invalid latitude values
pattern = r'^[A-Za-z0-9]+$'

# Create a boolean mask for rows to keep: rows that do not match the pattern and are not 'NULL' or 'N/A'
mask = ~df['lat'].str.contains(pattern, na=False) & ~df['lat'].isin(['NULL'])

# Filter the DataFrame using the mask
df = df[mask]

df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 441 entries, 0 to 450
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          441 non-null    int64 
 1   address        441 non-null    object
 2   longitude      441 non-null    object
 3   lat            1 non-null      object
 4   locality       441 non-null    object
 5   store_code     441 non-null    object
 6   staff_numbers  441 non-null    object
 7   opening_date   441 non-null    object
 8   store_type     441 non-null    object
 9   latitude       440 non-null    object
 10  country_code   441 non-null    object
 11  continent      441 non-null    object
dtypes: int64(1), object(11)
memory usage: 44.8+ KB


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 441 entries, 0 to 450
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          441 non-null    int64 
 1   address        441 non-null    object
 2   longitude      441 non-null    object
 3   lat            1 non-null      object
 4   locality       441 non-null    object
 5   store_code     441 non-null    object
 6   staff_numbers  441 non-null    object
 7   opening_date   441 non-null    object
 8   store_type     441 non-null    object
 9   latitude       440 non-null    object
 10  country_code   441 non-null    object
 11  continent      441 non-null    object
dtypes: int64(1), object(11)
memory usage: 44.8+ KB


In [17]:
# Check if 'locality' column contains any non-string values
non_string_values = df[~df['locality'].apply(lambda x: isinstance(x, str))]

# Display the non-string values
print("Rows with non-string values in 'locality' column:")
print(non_string_values)

Rows with non-string values in 'locality' column:
Empty DataFrame
Columns: [index, address, longitude, lat, locality, store_code, staff_numbers, opening_date, store_type, latitude, country_code, continent]
Index: []


In [19]:
df['locality'].unique()

array(['N/A', 'High Wycombe', 'Landshut', 'Westbury', 'Belper',
       'Gainsborough', 'Rutherglen', 'Stuttgart', 'Kaukauna', 'Hartley',
       'Siegburg', 'Devizes', 'Crystal Lake', 'Halstenbek', 'Lancing',
       'Newbury', 'Lymington', 'Chapletown', 'Barnet', 'East Dereham',
       'Poulton-le-Fylde', 'Bushey', 'Ehingen', 'Aberdeen', 'Surbiton',
       'Inverness', 'Weimar', 'Penzance', 'Albstadt', 'Hechingen',
       'Mexborough', 'Arbroath', 'Cowes', 'Clacton-on-Sea', 'Boston',
       'Charlottenburg-Nord', 'Bretten', 'Sun City Center', 'Karben',
       'Martinsburg', 'Oschatz', 'Kingston upon Hull', 'Leyland',
       'Venice', 'Schleswig', 'Eppingen', 'Losheim', 'Burscheid',
       'Southsea', 'Porterville', 'Walsrode', 'Exeter', 'Thetford',
       'Morningside Heights', 'Searcy', 'Radevormwald', 'Stade',
       'Mira Mesa', 'Strood', 'Verl', 'Wesseling', 'Pfullingen',
       'Bensheim', 'Walton-on-the-Naze', 'Friedberg', 'Winsford',
       'Newburgh', 'Selby', 'Oberhausen', 'Zei

In [26]:

# Find rows where 'locality' is 'N/A'
locality_na = df.loc[df['locality'] == 'N/A']

# Find rows where 'locality' is None or NaN
locality_none = df.loc[pd.isna(df['locality'])]

# Find rows where 'locality' is 'NULL'
locality_null = df.loc[df['locality'] == 'NULL']

# Display the results
print("Rows where 'locality' is 'N/A':")
print(locality_na)

print("\nRows where 'locality' is None or NaN:")
print(locality_none)

print("\nRows where 'locality' is 'NULL':")
display(locality_null)


Rows where 'locality' is 'N/A':
   index address longitude  lat locality    store_code staff_numbers  \
0      0     N/A       N/A  N/A      N/A  WEB-1388012W           325   

  opening_date  store_type latitude country_code continent  
0   2010-06-12  Web Portal     None           GB    Europe  

Rows where 'locality' is None or NaN:
Empty DataFrame
Columns: [index, address, longitude, lat, locality, store_code, staff_numbers, opening_date, store_type, latitude, country_code, continent]
Index: []

Rows where 'locality' is 'NULL':


,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent


In [44]:
#sandbox for 'lat'

# print df
print(df.info())

# create empty list for lat_items 
str_items = []
nonetype_items = []
empty_items = [] 
lat_items = []
null_str_items = [] 

# Go through the series 'lat' and append each item to lat_items
for x in df['lat']:
    if x is None: 
        #print('Nonetype:')
        nonetype_items.append(x)
    elif x == 'None':  # Check for string 'None'
        #print('string')
        str_items.append(x)
    elif x == 'NULL':
        #print('string')
        null_str_items.append(x)
    elif x == '':  # Check for empty string
        #print('empty string:')
        empty_items.append(x)
    else:
        #print('dont know')
        lat_items.append(x)

# Print out lat_items
print('lat', lat_items)
print('str:', str_items) 
print('nonetype', nonetype_items)
print('empty', empty_items)
print('null string:', null_str_items)

# drop na rows from the 'lat' column 
#df = df.dropna(subset=['lat'])

# # Define the regex pattern
# string_pattern = r'^[A-Za-z0-9]+$'

# # Fill NaN values with a placeholder that does not match the regex
# df['lat'] = df['lat'].fillna('@@@')

# # Filter rows where 'lat' matches the pattern
# df = df[df['lat'].str.match(string_pattern)]

# # Reset 'lat' column to NaN where the placeholder was used
# df['lat'] = df['lat'].replace('placeholder', np.nan)

# Go through the series 'lat' and append each item to lat_items
for x in df['lat']:
    if x is None: 
        #print('Nonetype:')
        nonetype_items.append(x)
    elif x == 'None':  # Check for string 'None'
        #print('string')
        str_items.append(x)
    elif x == 'NULL':
        #print('string')
        null_str_items.append(x)
    elif x == '':  # Check for empty string
        #print('empty string:')
        empty_items.append(x)
    else:
        #print('dont know')
        lat_items.append(x)

# Print out lat_items
print('lat', lat_items)
print('str:', str_items) 
print('nonetype', nonetype_items)
print('empty', empty_items)
print('null string:', null_str_items)


# go through the remaining lat items and add them to new_lat_items
# new_lat_items = [] 
# for x in df['lat']:
#     new_lat_items.append(x)

# print new lat items 
# print(new_lat_items)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          451 non-null    int64 
 1   address        451 non-null    object
 2   longitude      451 non-null    object
 3   lat            11 non-null     object
 4   locality       451 non-null    object
 5   store_code     451 non-null    object
 6   staff_numbers  451 non-null    object
 7   opening_date   451 non-null    object
 8   store_type     451 non-null    object
 9   latitude       450 non-null    object
 10  country_code   451 non-null    object
 11  continent      451 non-null    object
dtypes: int64(1), object(11)
memory usage: 42.4+ KB
None
lat ['N/A', '13KJZ890JH', '2XE1OWOC23', 'OXVE5QR07O', 'VKA5I8H32X', 'LACCWDI0SB', 'A3O5CBWAMD', 'UXMWDMX1LC']
str: []
nonetype [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 63 to 447
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          10 non-null     int64 
 1   address        10 non-null     object
 2   longitude      10 non-null     object
 3   lat            10 non-null     object
 4   locality       10 non-null     object
 5   store_code     10 non-null     object
 6   staff_numbers  10 non-null     object
 7   opening_date   10 non-null     object
 8   store_type     10 non-null     object
 9   latitude       10 non-null     object
 10  country_code   10 non-null     object
 11  continent      10 non-null     object
dtypes: int64(1), object(11)
memory usage: 1.0+ KB


In [25]:
#sandbox for latitde 
 
#df.info()
num_rows = df.shape[0]
print(f"Number of rows store data before cleaning: {num_rows}")

df['latitude']

print(type(df['latitude']))

# Create an empty list to store non-matching card numbers
non_matching = []

# Define a regular expression pattern for numbers only
pattern = r'^-?\d+(\.\d+)?$'

# Ensure all elements in the 'card_number' column are strings
df['latitude'] = df['latitude'].astype(str)

for x in df['latitude']:
    if not re.match(pattern, str(x)):
        non_matching.append(x)

# Print the list of non-matching card numbers
print("List of non-matching card numbers:", non_matching)
print(len(non_matching))

df = df[df['latitude'].apply(lambda x: bool(re.match(pattern, x)))]


# df = df.dropna(subset=['latitude'])
print(df.info())
num_rows = df.shape[0]
print(f"Number of rows store data after cleaning latitide column: {num_rows}")

Number of rows store data before cleaning: 451
<class 'pandas.core.series.Series'>
List of non-matching card numbers: ['None', '1CJ5OAU4BR', '3ZR3F89D97', 'NULL', 'L13EQEQODP', 'J3BPB68Z1J', 'AJHOMDOHZ4', 'NULL', 'AE7EEW4HSS', 'NULL', 'JZP8MIJTPZ']
11
<class 'pandas.core.frame.DataFrame'>
Index: 440 entries, 1 to 450
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          440 non-null    int64 
 1   address        440 non-null    object
 2   longitude      440 non-null    object
 3   lat            0 non-null      object
 4   locality       440 non-null    object
 5   store_code     440 non-null    object
 6   staff_numbers  440 non-null    object
 7   opening_date   440 non-null    object
 8   store_type     440 non-null    object
 9   latitude       440 non-null    object
 10  country_code   440 non-null    object
 11  continent      440 non-null    object
dtypes: int64(1), object(11)
memory usage: 44.

In [10]:
# code to clean the dataframe 
# Number of rows store data before cleaning: 451
# Number of rows store data after dropping lat and cleaning latitide column: 440
# Number of rows store data after cleaning locality: 440
# Number of rows store data after cleaning continent: 440
# Number of rows store data after cleaning opening date: 428
# Number of rows store data after cleaning: 428

df.info()
num_rows = df.shape[0]
print(f"Number of rows store data before cleaning: {num_rows}")

# Clean the latitude column


df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')

df = df.dropna(subset=['latitude'])

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning latitide column: {num_rows}")

# Dropping the 'latitude' column as it's empty 
df = df.drop(columns=['lat'])

num_rows = df.shape[0]
print(f"Number of rows store data after dropping lat column: {num_rows}")

# filtering out items in locality that aren't real place names or NULL 
pattern = r'^[a-zA-Z\s-]+$'
df = df[df['locality'].str.match(pattern)]
df['locality'] = df['locality'].replace('NULL', np.nan)
df = df.dropna(subset=['locality'])

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning locality: {num_rows}")

# replacing incorrect spellings of continents 
continent_replacements = {
    'eeEurope': 'Europe',
    'eeAmerica': 'America'
}

df['continent'] = df['continent'].replace(continent_replacements)

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning continent: {num_rows}")

# LOGGING: checking everything has worked 
#print(df['continent'].unique())
#print(df['store_type'].unique()) 
#print(df['country_code'].unique()) 
#print(df['continent'].unique())

# LOGGING: checking of datetime before is datetime64 datetype  
#is_datetime_before = pd.api.types.is_datetime64_any_dtype(df['opening_date'])

# converting opening date to datetime object 
df['opening_date'] = pd.to_datetime(df['opening_date'], format='%Y-%m-%d', errors='coerce')

# dropping any rows which contain missing values  
df = df.dropna(axis=0)

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning opening date: {num_rows}")

# LOGGING: checking the conversion worked 
#is_datetime_after = pd.api.types.is_datetime64_any_dtype(df['opening_date'])
#print(f"Is 'dates' column datetime64 dtype? {is_datetime_before}")
#print(f"Is 'dates' column datetime64 dtype? {is_datetime_after}")

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning: {num_rows}")
#returning the dataframe 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          451 non-null    int64 
 1   address        451 non-null    object
 2   longitude      451 non-null    object
 3   lat            11 non-null     object
 4   locality       451 non-null    object
 5   store_code     451 non-null    object
 6   staff_numbers  451 non-null    object
 7   opening_date   451 non-null    object
 8   store_type     451 non-null    object
 9   latitude       450 non-null    object
 10  country_code   451 non-null    object
 11  continent      451 non-null    object
dtypes: int64(1), object(11)
memory usage: 42.4+ KB
Number of rows store data before cleaning: 451
Number of rows store data after cleaning latitide column: 440
Number of rows store data after dropping lat column: 440
Number of rows store data after cleaning locality: 440
Number of rows store

In [ ]:

non_matching_cards = []
strings = []
numbers = []

for x in df['card_number']:
    if isinstance(x, str):
        strings.append(x)
    elif isinstance(x, int):
        numbers.append(x)
    else:
        non_matching_cards.append(x)